# AgentCore 런타임(Runtime)에서 미들웨어(Middleware) 지원

이 노트북은 Amazon Bedrock AgentCore 런타임(Runtime)에서 미들웨어(Middleware)를 구현하는 방법을 보여줍니다.

## 배울 내용

- Starlette/ASGI 애플리케이션에서 미들웨어(Middleware)가 작동하는 방식
- BaseHTTPMiddleware로 커스텀 미들웨어(Middleware) 생성
- BedrockAgentCoreApp에 미들웨어(Middleware) 전달
- 여러 미들웨어(Middleware) 컴포넌트 체이닝
- 미들웨어(Middleware) 활성화된 에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deployment)

## 사전 요구사항
- AWS 자격 증명 구성(Configuration)
- requirements.txt에서 필요한 패키지 설치
- Amazon Bedrock 콘솔에서 모델 액세스 활성화
- Docker 실행 중

## 설정

필요한 패키지를 설치하고 종속성을 임포트합니다.

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import time
from datetime import datetime

from bedrock_agentcore import BedrockAgentCoreApp
from starlette.middleware import Middleware
from starlette.middleware.base import BaseHTTPMiddleware

## Understanding Middleware

Middleware in AgentCore Runtime uses Starlette's ASGI middleware system. Middleware is evaluated from top-to-bottom:

```
Request → Middleware 1 → Middleware 2 → Agent → Middleware 2 → Middleware 1 → Response
```

### BaseHTTPMiddleware Pattern

To create middleware, you:

1. Inherit from `BaseHTTPMiddleware`
2. Implement `async def dispatch(self, request, call_next)`
3. Call `response = await call_next(request)` to invoke the next layer
4. Process the request before `call_next()` and the response after

The `call_next` function passes the request to the next middleware or your agent handler. You can inspect/modify the request before calling it, and inspect/modify the response after.

### Key Points

- Middleware wraps your application as ASGI components
- Each middleware can process requests and responses
- Middleware executes in the order specified in the list
- Use `request.state` to pass data between middleware and handlers
- Keep middleware stateless - don't store request-specific data in instance variables

## 미들웨어(Middleware) 이해

AgentCore 런타임(Runtime)의 미들웨어(Middleware)는 Starlette의 ASGI 미들웨어(Middleware) 시스템을 사용합니다. 미들웨어(Middleware)는 위에서 아래로 평가됩니다:

```
요청 → 미들웨어(Middleware) 1 → 미들웨어(Middleware) 2 → 에이전트(Agent) → 미들웨어(Middleware) 2 → 미들웨어(Middleware) 1 → 응답
```

### BaseHTTPMiddleware 패턴

미들웨어(Middleware)를 생성하려면:

1. `BaseHTTPMiddleware`를 상속
2. `async def dispatch(self, request, call_next)` 구현
3. `response = await call_next(request)`를 호출하여 다음 레이어 호출(Invocation)
4. `call_next()` 전에 요청을 처리하고 후에 응답을 처리

`call_next` 함수는 요청을 다음 미들웨어(Middleware) 또는 에이전트(Agent) 핸들러로 전달합니다. 호출(Invocation) 전에 요청을 검사/수정하고 후에 응답을 검사/수정할 수 있습니다.

### 핵심 포인트

- 미들웨어(Middleware)는 애플리케이션을 ASGI 컴포넌트로 래핑
- 각 미들웨어(Middleware)는 요청과 응답을 처리할 수 있음
- 미들웨어(Middleware)는 리스트에 지정된 순서대로 실행
- `request.state`를 사용하여 미들웨어(Middleware)와 핸들러 간에 데이터 전달
- 미들웨어(Middleware)를 무상태로 유지 - 인스턴스 변수에 요청별 데이터 저장하지 않음

## 일반적인 미들웨어(Middleware) 사용 사례

미들웨어(Middleware)를 구현하기 전에 일반적인 사용 사례를 이해해 보겠습니다:

### 1. 로깅 및 관측성
디버깅, 감사, 규정 준수를 위해 모든 요청과 응답을 추적합니다. 타임스탬프, 요청 세부정보, 응답 상태를 캡처하여 프로덕션에서 에이전트(Agent) 동작을 이해합니다.

### 2. 메트릭 수집
요청 지속 시간, 성공률, 오류 수와 같은 성능 메트릭을 측정합니다. SLA 모니터링, 병목 현상 식별, 에이전트(Agent) 성능 최적화에 필수적입니다.

### 3. 오류 처리 및 포맷팅
오류 응답을 표준화하고, 문제 해결을 위한 상관관계 ID를 추가하고, 오류를 사용자 친화적인 메시지로 변환합니다. 에이전트(Agent) 전체에서 일관된 오류 처리를 보장합니다.

### 4. 콘텐츠 필터링 및 가드레일(Guardrails)
Amazon Bedrock 가드레일(Guardrails)을 적용하여 에이전트(Agent)에 도달하기 전에 요청을 필터링합니다. 유해한 콘텐츠를 차단하고, 금지된 주제를 적용하며, 정책 위반을 방지합니다. 나쁜 요청을 일찍 거부하여 컴퓨팅 비용을 절약합니다.

### 5. 속도 제한
남용을 방지하고 공정한 리소스 할당을 보장하기 위해 사용자 또는 API 키당 요청 빈도를 제어합니다. 과부하로부터 에이전트(Agent)를 보호하고 비용을 관리합니다.

### 6. 인증(Authentication) 및 인가(Authorization)
요청이 에이전트(Agent)에 도달하기 전에 API 키, JWT 토큰 또는 커스텀 자격 증명을 검증합니다. 에이전트(Agent) 로직을 복잡하게 만들지 않고 보안을 추가합니다.

이 튜토리얼에서는 **로깅 및 메트릭**(결합), **오류 처리**, **가드레일(Guardrails)을 사용한 콘텐츠 필터링**을 위한 미들웨어(Middleware)를 구현하여 가장 일반적인 프로덕션 패턴을 시연합니다.

In [ ]:
# Key concept: Use async dispatch to wrap request/response
class ObservabilityMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        # Before request: Log and start timer
        print(f"REQUEST: {request.method} {request.url.path}")
        start_time = time.time()
        
        # Process request
        response = await call_next(request)
        
        # After response: Log duration
        duration = time.time() - start_time
        print(f"RESPONSE: Status {response.status_code} | Duration {duration:.4f}s")
        
        return response



## 예제 1: 관측성 미들웨어(Middleware) (로깅 + 메트릭)

이 미들웨어(Middleware)는 로깅과 메트릭 수집 - 밀접하게 관련된 두 가지 관심사를 결합합니다:

**로깅:**
- 요청 메서드, 경로, 타임스탬프 기록
- 응답 상태 및 지속 시간 로깅
- 규정 준수를 위한 감사 추적 제공

**메트릭:**
- 요청 처리 시간 측정
- 요청 수 및 패턴 추적
- 성능 모니터링 활성화


**왜 결합하나요?** 둘 다 타이밍을 측정하고 동일한 요청/응답 데이터에 액세스해야 합니다. 결합하면 오버헤드가 줄어들고 관련 기능이 함께 유지됩니다.

In [ ]:
# Key concept: Wrap in try/except to catch errors
class ErrorHandlingMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        correlation_id = str(uuid.uuid4())
        
        try:
            response = await call_next(request)
            response.headers["x-correlation-id"] = correlation_id
            return response
        except Exception as e:
            # Log error with correlation ID
            print(f"ERROR: {type(e).__name__}: {str(e)}")
            print(f"Correlation ID: {correlation_id}")
            
            # Return user-friendly error
            return JSONResponse(
                status_code=500,
                content={"error": "An error occurred", "correlation_id": correlation_id}
            )



## Creating a Production Agent with Middleware

Let's create an agent file with Observability and Error Handling middleware.

In [ ]:
%%writefile middleware_agent.py
import time
import json
from datetime import datetime
import traceback
import uuid

from bedrock_agentcore import BedrockAgentCoreApp
from starlette.middleware import Middleware
from starlette.middleware.base import BaseHTTPMiddleware
from starlette.responses import JSONResponse
from strands import Agent
from strands.models import BedrockModel

# Middleware 1: Observability
class ObservabilityMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        timestamp = datetime.now().isoformat()
        print(f"\n[{timestamp}] REQUEST: {request.method} {request.url.path}")
        start_time = time.time()
        response = await call_next(request)
        duration = time.time() - start_time
        print(f"[{timestamp}] RESPONSE: Status {response.status_code} | Duration {duration:.4f}s")
        response.headers["x-process-time"] = f"{duration:.4f}s"
        return response

# Middleware 2: Error Handling
class ErrorHandlingMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        correlation_id = str(uuid.uuid4())
        try:
            response = await call_next(request)
            response.headers["x-correlation-id"] = correlation_id
            return response
        except Exception as e:
            print(f"\n❌ ERROR: {type(e).__name__}: {str(e)}")
            return JSONResponse(
                status_code=500,
                content={"error": "An error occurred", "correlation_id": correlation_id}
            )

# Create app with middleware
app = BedrockAgentCoreApp(
    middleware=[
        Middleware(ErrorHandlingMiddleware),
        Middleware(ObservabilityMiddleware),
    ]
)

# Initialize agent
model = BedrockModel(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0")
agent = Agent(model=model, system_prompt="You are a helpful AI assistant.")

@app.entrypoint
def agent_handler(payload, context):
    user_message = payload.get("prompt", "Hello!")
    result = agent(user_message)
    return {"response": result.message}

if __name__ == "__main__":
    app.run()

## 예제 2: 오류 처리 미들웨어(Middleware)

이 미들웨어(Middleware)는 에이전트(Agent) 전체에서 오류 처리를 표준화합니다:

**오류 처리:**
- 에이전트(Agent)가 충돌하기 전에 예외 포착
- 디버깅을 위한 전체 컨텍스트와 함께 오류 로깅
- 민감한 오류 세부정보가 클라이언트에 유출되는 것 방지

**오류 포맷팅:**
- 일관된 오류 응답 구조 반환
- 문제 해결을 위한 상관관계 ID 추가
- 사용자 친화적인 오류 메시지 제공

**왜 중요한가요:** 오류 미들웨어(Middleware) 없이는 예외가 내부 세부정보를 노출하거나, 에이전트(Agent)를 충돌시키거나, 일관되지 않은 오류 형식을 반환할 수 있습니다. 이 미들웨어(Middleware)는 우아한 저하와 더 나은 디버깅을 보장합니다.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="middleware_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="middleware_agent"
)

print("✓ Agent configured")

In [ ]:
# Launch the agent
launch_result = agentcore_runtime.launch()
print("✓ Agent launched")

In [ ]:
# Wait for deployment to complete
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']

print(f"\n✓ Deployment complete: {status}")

## Test the Agent

Let's test our agent with basic middleware.

In [ ]:
import json

response = agentcore_runtime.invoke({
    "prompt": "What is the capital of France?"
})

response_data = json.loads(response['response'][0])
print("Agent Response:")
print("="*60)
print(response_data['response'])

print("\n" + "="*60)
print("📊 View Middleware Output in CloudWatch Logs:")
print("="*60)
print("   /aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/[runtime-logs]")
print("\nYou'll see:")
print("  - REQUEST: timestamp, method, path")
print("  - RESPONSE: status code, duration")
print("  - Correlation IDs for tracking")

## 미들웨어(Middleware)를 사용한 프로덕션 에이전트(Agent) 생성

관측성과 오류 처리 미들웨어(Middleware)가 있는 에이전트(Agent) 파일을 생성해 보겠습니다.

In [ ]:
import boto3

bedrock_client = boto3.client('bedrock', region_name='us-east-1')

# Create guardrail - only name, blockedInputMessaging, and blockedOutputsMessaging are required
guardrail_response = bedrock_client.create_guardrail(
    name='financial-advice-blocker',
    description='Blocks requests for financial advice',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Financial Advice',
                'definition': 'Questions seeking investment advice, stock recommendations, or financial planning guidance',
                'examples': [
                    'Should I invest in cryptocurrency?',
                    'What stocks should I buy?',
                    'How should I plan my retirement savings?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {'type': 'HATE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'MISCONDUCT', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'PROMPT_ATTACK', 'inputStrength': 'HIGH', 'outputStrength': 'NONE'}
        ]
    },
    blockedInputMessaging='This request violates our content policy.',
    blockedOutputsMessaging='This response violates our content policy.'
)

guardrail_id = guardrail_response['guardrailId']
guardrail_version = guardrail_response['version']
print(f"✓ Created guardrail: {guardrail_id} (version: {guardrail_version})")

### Guardrail Middleware Concept

This middleware checks requests before they reach your agent and blocks policy violations.

In [ ]:
# Key concept: Check request before passing to agent
class GuardrailMiddleware(BaseHTTPMiddleware):
    def __init__(self, app, guardrail_id: str):
        super().__init__(app)
        self.guardrail_id = guardrail_id
        self.bedrock_runtime = boto3.client('bedrock-runtime')
    
    async def dispatch(self, request, call_next):
        if request.method == 'POST' and '/invocations' in request.url.path:
            body = await request.body()
            payload = json.loads(body)
            user_prompt = payload.get('prompt', '')
            
            # Apply guardrail
            result = self.bedrock_runtime.apply_guardrail(
                guardrailIdentifier=self.guardrail_id,
                guardrailVersion='DRAFT',
                source='INPUT',
                content=[{'text': {'text': user_prompt}}]
            )
            
            # Block if guardrail intervened - return agent-compatible response
            if result['action'] == 'GUARDRAIL_INTERVENED':
                blocked_msg = result['outputs'][0]['text'] if result.get('outputs') else 'Content blocked by guardrail'
                print(f"🛡️ Guardrail blocked: {user_prompt[:50]}...")
                
                # Return in agent response format
                return JSONResponse(
                    status_code=200,
                    content={'response': blocked_msg}
                )
        
        return await call_next(request)



## AgentCore 런타임(Runtime)에 배포(Deployment)

관측성과 오류 처리 미들웨어(Middleware)가 있는 에이전트(Agent)를 배포(Deployment)해 보겠습니다.

In [ ]:
%%writefile middleware_agent.py
import time
import json
from datetime import datetime
import traceback
import uuid
import os

import boto3
from bedrock_agentcore import BedrockAgentCoreApp
from starlette.middleware import Middleware
from starlette.middleware.base import BaseHTTPMiddleware
from starlette.responses import JSONResponse
from strands import Agent
from strands.models import BedrockModel

# Middleware 1: Observability
class ObservabilityMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        timestamp = datetime.now().isoformat()
        print(f"\n[{timestamp}] REQUEST: {request.method} {request.url.path}")
        start_time = time.time()
        response = await call_next(request)
        duration = time.time() - start_time
        print(f"[{timestamp}] RESPONSE: Status {response.status_code} | Duration {duration:.4f}s")
        response.headers["x-process-time"] = f"{duration:.4f}s"
        return response

# Middleware 2: Error Handling
class ErrorHandlingMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        correlation_id = str(uuid.uuid4())
        try:
            response = await call_next(request)
            response.headers["x-correlation-id"] = correlation_id
            return response
        except Exception as e:
            print(f"\n❌ ERROR: {type(e).__name__}: {str(e)}")
            return JSONResponse(
                status_code=500,
                content={"error": "An error occurred", "correlation_id": correlation_id}
            )

# Middleware 3: Guardrail (NEW!)
class GuardrailMiddleware(BaseHTTPMiddleware):
    def __init__(self, app, guardrail_id: str, guardrail_version: str = 'DRAFT'):
        super().__init__(app)
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.bedrock_runtime = boto3.client('bedrock-runtime')
    
    async def dispatch(self, request, call_next):
        if request.method == 'POST' and '/invocations' in request.url.path:
            body = await request.body()
            try:
                payload = json.loads(body)
                user_prompt = payload.get('prompt', '')
                
                result = self.bedrock_runtime.apply_guardrail(
                    guardrailIdentifier=self.guardrail_id,
                    guardrailVersion=self.guardrail_version,
                    source='INPUT',
                    content=[{'text': {'text': user_prompt}}]
                )
                
                if result['action'] == 'GUARDRAIL_INTERVENED':
                    blocked_msg = result['outputs'][0]['text'] if result.get('outputs') else 'Content blocked'
                    print(f"🛡️ Guardrail blocked: {user_prompt[:50]}...")
                    return JSONResponse(status_code=200, content={'response': blocked_msg})
                
                print(f"✓ Guardrail passed: {user_prompt[:50]}...")
            except Exception as e:
                print(f"⚠️ Guardrail check failed: {e}")
        
        return await call_next(request)

# Get guardrail ID from environment
GUARDRAIL_ID = os.environ.get('GUARDRAIL_ID', '')

# Build middleware list
middleware_list = [
    Middleware(ErrorHandlingMiddleware),
    Middleware(ObservabilityMiddleware),
]

if GUARDRAIL_ID:
    middleware_list.insert(0, Middleware(GuardrailMiddleware, guardrail_id=GUARDRAIL_ID))
    print(f"✓ Guardrail enabled: {GUARDRAIL_ID}")

app = BedrockAgentCoreApp(middleware=middleware_list)

# Initialize agent
model = BedrockModel(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0")
agent = Agent(model=model, system_prompt="You are a helpful AI assistant.")

@app.entrypoint
def agent_handler(payload, context):
    user_message = payload.get("prompt", "Hello!")
    result = agent(user_message)
    return {"response": result.message}

if __name__ == "__main__":
    app.run()

In [ ]:
# Update the existing agent with guardrail
launch_result = agentcore_runtime.launch(
    env_vars={'GUARDRAIL_ID': guardrail_id},
    auto_update_on_conflict=True
)

# Wait for update
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']

print(f"\n✓ Agent updated with guardrail: {status}")

### Test Guardrail Middleware

Now let's test with a financial advice question (blocked) and a normal question (allowed).

In [ ]:
# Test 1: Financial advice (should be blocked)
print("Test 1: Financial Advice Question (should be blocked)")
print("="*60)

response = agentcore_runtime.invoke({
    "prompt": "Should I invest in cryptocurrency? What stocks should I buy?"
})
response_data = json.loads(response['response'][0])
print(f"Response: {response_data['response']}")

print("\n" + "="*60)
print("\nTest 2: Normal Question (should pass)")
print("="*60)

response = agentcore_runtime.invoke({
    "prompt": "What is the capital of France?"
})
response_data = json.loads(response['response'][0])
print(f"Response: {response_data['response']}")

print("\n" + "="*60)
print("📊 Check CloudWatch Logs to see:")
print("  - Guardrail blocking financial advice")
print("  - Guardrail passing normal questions")
print("  - Request timing and correlation IDs")

## Cleanup

When you're done, clean up the deployed resources.

In [ ]:
agentcore_runtime.destroy()

## Summary

You learned:

✓ How middleware works in AgentCore Runtime using Starlette's ASGI system  
✓ Creating custom middleware with BaseHTTPMiddleware  
✓ Common patterns: logging, metrics, error handling, and content filtering  
✓ Integrating Amazon Bedrock Guardrails for request filtering  
✓ Chaining multiple middleware components  
✓ Deploying and testing middleware-enabled agents on AgentCore Runtime  

### Key Takeaways

- Middleware provides clean separation of cross-cutting concerns
- Middleware executes in order: first in list wraps all others
- Use `call_next()` to pass control to the next layer
- Guardrails can block requests early, before agent processing
- Keep middleware stateless for concurrent request handling
- Middleware logs appear in CloudWatch Logs for deployed agents